In [2]:
#importation des librairies python
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k
import time

In [ ]:
#debut chonometrage
start = time.time()

In [3]:
#importation des donnees
transactions = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\transactions_train.csv/transactions.csv', dtype={'article_id': str}, parse_dates=['t_dat'])
df_sub = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\sample_submission.csv/sample_submission.csv')
df_utilisateurs = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\customers.csv/customers.csv')
df_articles = pd.read_csv(r'C:\Users\Stagiaire\Downloads\Timsoft\H&M\articles.csv/articles.csv', dtype={'article_id': str})

Wall time: 39.8 s


In [6]:
#choix des transactions entre les dates 20-09-2018 & 20-09-2019
transactions = transactions[(transactions['t_dat'] > '2018-09-20') & (transactions['t_dat'] < '2019-09-20')]

,t_dat,customer_id,article_id,price,sales_channel_id
48399,2018-09-21,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0583558001,0.067780,2
48400,2018-09-21,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0639677008,0.025407,2
48401,2018-09-21,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0640244003,0.033881,2
48402,2018-09-21,0006d37aaf7dd84f9bbc02f6cadcb74fd72ebf370bdc5f...,0685687004,0.016932,2
48403,2018-09-21,0006d37aaf7dd84f9bbc02f6cadcb74fd72ebf370bdc5f...,0591334004,0.010153,2
...,...,...,...,...,...
16803896,2019-09-19,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,0764525002,0.015237,2
16803897,2019-09-19,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,0764525002,0.015237,2
16803898,2019-09-19,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,0703558001,0.022017,2
16803899,2019-09-19,fff98edc27fc5d64c3027bf0e3702510143d1a79c3dc9a...,0703558001,0.022017,2


In [8]:
#pour la validation 3 semaines de donnees pour l entrainement et 1 semaine de donnes pour la validation
transactions['t_dat'].max()
ALL_USERS = df_utilisateurs['customer_id'].unique().tolist()
ALL_ITEMS = df_articles['article_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

transactions['user_id'] = transactions['customer_id'].map(user_map)
transactions['item_id'] = transactions['article_id'].map(item_map)

del df_utilisateurs, df_articles

In [9]:
#construction de la matrice coo (matrice qui represente la matrice creuse des achats des utilisateurs)
row = transactions['user_id'].values
col = transactions['item_id'].values
data = np.ones(transactions.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

<1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 16755502 stored elements in COOrdinate format>

In [10]:
#voir si le modele marche avec les donnes proposes
#choix de l'algorithme ALS implicit
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(coo_train)

C:\Users\Stagiaire\anaconda3\lib\site-packages\implicit\utils.py:31: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

Wall time: 7.33 s


In [11]:
#Transforme un dataframme de transaction en une matrice creuse COO
def utilisateur_article_COO(df):
    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo

#creation des dataframe d entrainement et de validation
def div_data(df, validation_days=7):
    validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days)
    df_train = df[df['t_dat'] < validation_cut]
    df_val = df[df['t_dat'] >= validation_cut]
    return df_train, df_val

#creation des donnes d entrainement et de validation et plusieurs autres matrices (coo_train, crs_train, crs_val)
def PR_val_matrices(df, validation_days=7):

    df_train, df_val = div_data(df, validation_days=validation_days)
    coo_train = utilisateur_article_COO(df_train)
    coo_val = utilisateur_article_COO(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }

#entrainer un model ALS avec les facteurs latents pour un nombre d iterations sur des matrice et valide par la metrique MAP@12
def validate(matrices, factors=100, iterations=20, regularization=0.01, show_progress=True):
    
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [12]:
matrices = PR_val_matrices(transactions)

In [13]:
#phase d entrainement : les facteurs latents de 40 a 1000 les iteration de 3 a 20, regularisation 0.01 
%%time
best_map12 = 0
for factors in [40, 50, 60, 100, 200, 500, 1000]:
    for iterations in [3, 12, 14, 15, 20]:
        for regularization in [0.01]:
            map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
            if map12 > best_map12:
                best_map12 = map12
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
                print(f"Best MAP@12 found. Updating: {best_params}")

Factors:  40 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.00378
Best MAP@12 found. Updating: {'factors': 40, 'iterations': 3, 'regularization': 0.01}
Factors:  40 - Iterations: 12 - Regularization: 0.010 ==> MAP@12: 0.00403
Best MAP@12 found. Updating: {'factors': 40, 'iterations': 12, 'regularization': 0.01}
Factors:  40 - Iterations: 14 - Regularization: 0.010 ==> MAP@12: 0.00406
Best MAP@12 found. Updating: {'factors': 40, 'iterations': 14, 'regularization': 0.01}
Factors:  40 - Iterations: 15 - Regularization: 0.010 ==> MAP@12: 0.00402
Factors:  40 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.00402
Factors:  50 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.00397
Factors:  50 - Iterations: 12 - Regularization: 0.010 ==> MAP@12: 0.00420
Best MAP@12 found. Updating: {'factors': 50, 'iterations': 12, 'regularization': 0.01}
Factors:  50 - Iterations: 14 - Regularization: 0.010 ==> MAP@12: 0.00421
Best MAP@12 found. Updating: {'factors': 50, 'iterations': 14

In [14]:
del matrices

In [15]:
#entrainement sur tous le dataset
coo_train = utilisateur_article_COO(transactions)
csr_train = coo_train.tocsr()

In [16]:
def train(coo_train, factors=200, iterations=15, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    return model

In [18]:
#entrainement du model sur les meilleurs parametres 
model = train(coo_train, **best_params)

  0%|          | 0/20 [00:00<?, ?it/s]

In [19]:
#fonction qui a en sortie les recommandations
def submit(model, csr_train, submission_name="submissions.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx : startidx + batch_size]
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    df_preds.to_csv(submission_name, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [20]:
#construction du tableau de predictions
%%time
df_preds = submit(model, csr_train);

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601006 0568597006 0568601007 0568597007 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0351484002 0723529001 0458543001 0673677002 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0529012025 0611415001 0579302001 0611415005 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0564786001 0680262001 0484398001 0680263003 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0629758005 0590928013 0712924003 0698286003 07...


(1371980, 2)
Wall time: 6min 3s


In [22]:
#exportation des recommandations
df_preds.to_csv('Predictions.csv')

In [23]:
#fin du chronometrage
end = time.time()
print(end - start)

18897.161586284637
